In [21]:
!pip install -U langchain-ollama langchain-openai

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
!ls /content/drive/MyDrive/review_dataset

sample_submission.csv  w_review_train.csv
test_file.csv	       w_review_train.parquet


In [24]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [25]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [26]:
df.shape

(40000, 2)

In [27]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [28]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [29]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [30]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [31]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [32]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4.05 s, sys: 33.3 ms, total: 4.08 s
Wall time: 3.72 s


In [33]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [34]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.37 s, sys: 70.7 ms, total: 2.44 s
Wall time: 4.03 s


In [35]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [36]:
coffee_reviews_df.shape

(4981, 2)

In [37]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [38]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [39]:
!ollama pull gemma3:270m

In [40]:
!ollama list

NAME           ID              SIZE      MODIFIED               
gemma3:270m    e7d36fb2c3b3    291 MB    Less than a second ago    


In [41]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ ยินดีที่ได้รู้จักนะคะ 😊




In [42]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-20T08:13:26.201635481Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}}]}

In [43]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [44]:
llm = OllamaLLM(model="gemma3:270m")

In [45]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ ยินดีที่ได้รู้จักนะคะ 😊



In [46]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [47]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [48]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [49]:
chain = prompt | llm

In [50]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [51]:
response

'```python\ndef find_sum(numbers):\n    """\n    หาผลรวมของตัวเลข 1 ถึง 10\n    """\n    total = 0\n    for number in numbers:\n        total += number\n    return total\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nsum_of_numbers = find_sum(numbers)\nprint(sum_of_numbers)\n```\n\n**คำอธิบายโค้ด:**\n\n1.  **`def find_sum(numbers):`**:  กำหนดชื่อฟังก์ชัน `find_sum` ที่จะทำงาน ซึ่งจะ return value จะเป็นค่าที่เก็บผลรวมของตัวเลข 1 ถึง 10\n2.  **`total = 0`**: Initialize counter `total` to 0. This variable will store the sum of the numbers we\'re looking for.\n3.  **`for number in numbers:`**:  Iterate through each number in the `numbers` list.\n4.  **`total += number`**:  Add the current `number` to the `total` variable. This is equivalent to `total = total + number`.\n5.  **`return total`**:  After iterating through all the numbers, return the final `total`, which represents the sum of all the numbers.\n\n**การใช้งานโค้ด:**\n\n1.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8,

In [52]:
from IPython.display import display, Markdown

In [53]:
display(Markdown(response))

```python
def find_sum(numbers):
    """
    หาผลรวมของตัวเลข 1 ถึง 10
    """
    total = 0
    for number in numbers:
        total += number
    return total

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
sum_of_numbers = find_sum(numbers)
print(sum_of_numbers)
```

**คำอธิบายโค้ด:**

1.  **`def find_sum(numbers):`**:  กำหนดชื่อฟังก์ชัน `find_sum` ที่จะทำงาน ซึ่งจะ return value จะเป็นค่าที่เก็บผลรวมของตัวเลข 1 ถึง 10
2.  **`total = 0`**: Initialize counter `total` to 0. This variable will store the sum of the numbers we're looking for.
3.  **`for number in numbers:`**:  Iterate through each number in the `numbers` list.
4.  **`total += number`**:  Add the current `number` to the `total` variable. This is equivalent to `total = total + number`.
5.  **`return total`**:  After iterating through all the numbers, return the final `total`, which represents the sum of all the numbers.

**การใช้งานโค้ด:**

1.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  สร้าง list ของตัวเลขที่ต้องการหาผลรวม
2.  **`sum_of_numbers = find_sum(numbers)`**:  เรียกใช้ฟังก์ชัน `find_sum` เพื่อหาผลรวมของตัวเลขใน list
3.  **`print(sum_of_numbers)`**:  แสดงผลลัพธ์ของฟังก์ชัน

**ข้อดีของโค้ดนี้:**

*   **อ่านง่าย:**  โค้ดนี้เป็นภาษา Python ที่เข้าใจง่าย ทำให้ง่ายต่อการอ่านและใช้งาน
*   **ใช้ `for` loop:**  ใช้ loop `for` ที่เป็นมาตรฐานใน Python ทำให้การคำนวณเป็นไปอย่างมีประสิทธิภาพ
*   **มีประสิทธิภาพ:**  การคำนวณผลรวมของตัวเลขเป็นไปอย่างมีประสิทธิภาพ
*   **สามารถปรับแต่งได้:**  คุณสามารถปรับแต่งฟังก์ชัน `find_sum` ได้โดยการเพิ่มหรือลดค่า `total`  เพื่อให้ได้ผลลัพธ์ที่ตรงกับความต้องการ

**ตัวอย่างการปรับแก้:**

*   **เพิ่มค่า `total`:**  ถ้าคุณต้องการหาผลรวมของตัวเลขที่ 1 ถึง 10  โดยประมาณ  `total = 10 + 3`  จะทำให้ได้ผลลัพธ์ที่ถูกต้อง
*   **เพิ่มค่า `total`:**  ถ้าคุณต้องการหาผลรวมของตัวเลข 1 ถึง 10  โดยประมาณ  `total = 10 + 3`  จะทำให้ได้ผลลัพธ์ที่ถูกต้อง
*   **เพิ่มค่า `total`:**  ถ้าคุณต้องการหาผลรวมของตัวเลข 1 ถึง 10  โดยประมาณ  `total = 10 + 3`  จะทำให้ได้ผลลัพธ์ที่ถูกต้อง


In [54]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [55]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [56]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [57]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [58]:
parser_chain = StrOutputParser()

In [59]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [60]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [61]:
result

'แน่นอนครับ! ผมเข้าใจโจทย์ได้ถูกต้องแล้วครับ โดยเฉพาะอย่างยิ่งในบริบทของผลลัทธ์ครับ'

In [62]:
prompt_template = """
[1. Role/Context บอก LLM ว่าควรทำตัวเป็นอะไร]
คุณเป็น AI ที่เชี่ยวชาญในการวิเคราะห์รีวิวภาษาไทย

[2. Task Description บอกงานที่ต้องทำ]
จาก Review ต่อไปนี้ ให้ Extract 3 keywords
ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

[3. Input Data ข้อมูลที่ต้องประมวลผล]
Review: {review}

[4. Output Format บอกรูปแบบของคำตอบ]
ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}

[5. Constraints ข้อจำกัดหรือเงื่อนไข]
- ต้องเป็นคำภาษาไทย
- คั่นด้วย comma
- หากไม่พบให้ตอบ 'ไม่ระบุ'

[6. Examples (optional) ตัวอย่าง input/output]
ตัวอย่าง
Input: "กาแฟอร่อย บรรยากาศดี ราคาแพง"
Output: {"keywords": "กาแฟ, บรรยากาศ, ราคา"}
"""

In [63]:
prompt = """
Extract keywords from this review
{review}
"""

# Pros Simple, fast
# Cons อาจได้ผลไม่ดีกับ Tasks ซับซ้อน

In [64]:
prompt = """
Extract keywords from reviews.

Example 1
Review: "กาแฟอร่อย บริการดี"
Keywords: กาแฟ, บริการ

Example 2
Review: "ราคาแพง แต่คุ้มค่า"
Keywords: ราคา, คุณภาพ

Now extract from this review
Review: {review}
Keywords:
"""

# Pros ได้ผลดีกว่า Zero-shot
# Cons Prompt ใช้ Tokens มากขึ้น

In [65]:
prompt = """
Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
"""

# Pros ได้ผลดีกับ Reasoning tasks
# Cons ใช้ Tokens มากกว่า ช้ากว่า

In [66]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [67]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [68]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [69]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [70]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 74.7 ms, sys: 7.68 ms, total: 82.4 ms
Wall time: 9.98 s


In [71]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{"keywords": ["ร้านอาหาร", "ห้องน้ำ", "กาแฟ", "น้ำผึ้ง", "น้ำผึ้ง", "ลาบไข่ต้ม", "ไข่มัน", "ขนมไทย", "ขนมหวาน", "ฟรี", "ราคา", "กินไม่ไหว", "ขนมไทย", "ไข่อร่อย", "เห้ย", "แต่ราคา", "เกินไป", "รับไม่ไหว", "ว", "ไม่ไปซ้ำ", "แพงเกิน"]}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "กาแฟ",
    "ร้านวาวี",
    "อารีย์",
    "Starbuck",
    "แฟนๆ Starbuck",
    "กาแฟ",
    "อร่อย",
    "ระดับแฟนๆ Starbuck",
    "ดอง",
    "มาหา",
    "จิบ",
    "ฟังเพลง",
    "พักผ่อน",
    "ดี",
    "ราคาไม่แพง",
    "กินบ่อย",
    "กินกาแฟ",
    "จุใจ",
    "อร่อย",
    "มี",
    "แกล้ง",
    "จุกได้",
    "แกล้ง",
    "บ่อยๆ",
    "ตาม",
    "รอ",
    "โต๊ะ",
    "เสริฟ",
    "คน",
    "หลุดริน",
    "สตาฟーム",
    "อาหาร",
    "อาหาร",
    "ราคา",
    "กินบ่อยๆ",
    "จุก",
    "แ

In [72]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [73]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    Less than a second ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    About a minute ago        


In [74]:
llm3 = OllamaLLM(model="scb10x/typhoon2.1-gemma3-4b")

In [75]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [76]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 101 ms, sys: 15.2 ms, total: 116 ms
Wall time: 17.8 s


In [77]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text แน่นอนครับ ผมจะทำตามขั้นตอนที่คุณระบุมาเพื่อดึงคำสำคัญจากรีวิวนี้ และนำเสนอในรูปแบบ JSON ดังนี้ครับ

ขั้นตอนที่ 1: อ่านรีวิวและเข้าใจความหมาย

รีวิวนี้เป็นการรีวิวร้านอาหารที่มีขนาดใหญ่ มีข้อดีและข้อเสียหลายอย่าง เช่น อาหารบางอย่างอร่อย (ทอดมันหัวปลี, พะแนงห่อไข่) แต่มีข้อเสียคือราคาสูงเกินไป และอาหารบางอย่างไม่ถูกใจ (ลาบไข่ต้ม)

ขั้นตอนที่ 2: ระบุหัวข้อหลักที่กล่าวถึง

*   ขนาดร้าน
*   ราคา
*   รสชาติอาหาร (โดยเฉพาะทอดมันหัวปลีและพะแนงห่อไข่)
*   ขนมหวานฟรี

ขั้นตอนที่ 3: เลือก 3 keywords ที่สำคัญที่สุด

1.  ราคา
2.  รสชาติ
3.  ขนาด

ขั้นตอนที่ 4: ตอบในรูปแบบ JSON

```json
{
  "keywords": ["ราคา", "รสชาติ", "ขนาด"]
}
```

หวังว่าคำตอบนี้จะเป็นประโยชน์นะครับ หากคุณมีคำถามเพิ่มเติม หรือต้องการให้ผมช่วยอะไรอีก สามารถบอกได้เลยครับ

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text แน่นอนครับ ผมจะทำตามขั้นตอนที

In [78]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [79]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [80]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [81]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 142 ms, sys: 11.7 ms, total: 154 ms
Wall time: 13 s


In [82]:
results[0]['text']['keywords']

['ราคา', 'อาหาร', 'ขนาด']

In [83]:
results[1]['text']['keywords']

['วาวี', 'กาแฟ', 'อารีย์']

In [84]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [85]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 137 ms, sys: 6.5 ms, total: 143 ms
Wall time: 12.9 s


In [86]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'อาหาร', 'ขนาด'], ['กาแฟ', 'บรรยากาศ', 'รสชาติ'])

In [87]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [88]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [89]:
inputs = [{"review": r} for r in sample_reviews]

In [90]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 17.9 ms, sys: 3.12 ms, total: 21.1 ms
Wall time: 2.4 s


In [91]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [92]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[0:200].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]

BATCH_SIZE = 8

results_100 = []

In [95]:
from tqdm import tqdm

for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 25/25 [04:26<00:00, 10.68s/it]


In [96]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part1.csv", index=False, encoding='utf-8-sig')

In [97]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[200:400].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]
BATCH_SIZE = 8
results_100 = []

In [98]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 25/25 [04:12<00:00, 10.11s/it]


In [99]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part2.csv", index=False, encoding='utf-8-sig')

In [100]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[400:600].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]
BATCH_SIZE = 8
results_100 = []

In [101]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 25/25 [04:05<00:00,  9.84s/it]


In [102]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part3.csv", index=False, encoding='utf-8-sig')

In [103]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[600:800].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]
BATCH_SIZE = 8
results_100 = []

In [104]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 25/25 [04:09<00:00,  9.98s/it]


In [105]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part4.csv", index=False, encoding='utf-8-sig')

In [106]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[800:1000].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]
BATCH_SIZE = 8
results_100 = []

In [107]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 25/25 [03:59<00:00,  9.59s/it]


In [108]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part5.csv", index=False, encoding='utf-8-sig')

In [94]:
#from tqdm import tqdm

In [109]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 25/25 [03:59<00:00,  9.58s/it]


In [110]:
for result in results_100[0:10]:
    print(result['text']['keywords'])

ช็อกโกแลต, เค้ก, ขนมหวาน
สนามบิน, ร้านอาหาร, บรรยากาศ
กาแฟ, เค้ก, บรรยากาศ
อาหารอิตาเลียน, บรรยากาศ, พิซซ่า
บรรยากาศ, ฟิวชั่น, ราคา
บรรยากาศ, ส้มตำ, รสชาติ
กาแฟ, เอ็กซ์เพรสโซ่, กลิ่น
วาฟเฟิล, กาแฟ, ไซรัป
กาแฟ, โปรโมชั่น, คนเยอะ
อาหารฝรั่ง, ชีสทอด, ฮันนี่โทส


In [111]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [112]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)


In [113]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [114]:
context_extract = context_prompt | llm3 | context_parser

In [115]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [116]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [117]:
results_100[0]['text']['keywords'].split(',')

['ช็อกโกแลต', ' เค้ก', ' ขนมหวาน']

In [118]:
context_inputs = []

for review_data in tqdm(results_100, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords'].split(',')
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 400/400 [00:00<00:00, 168835.83it/s]


In [119]:
len(context_inputs)

1200

In [120]:
context_results = []
for i in tqdm(range(len(results_100)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_100[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 400/400 [28:40<00:00,  4.30s/it]


In [121]:
results_100[:2]

[{'review': 'Chocolate crape cake with strawberry 145฿\nDark chocolate Ganache cake 145฿\n\nบริการแบบ งงๆ แบบมีหลายเคาร์เตอร์ \n\nขนมอร่อย ร้านตกแต่งสวย มีทั้งอาหาร ขนม กาแฟ มีของ grocery ให้ซื้อไป cook เองที่บ้านได้ \n\nอิ่มคาวมาจากร้านข้างบนแล้ว มาต่อแค่ขนมหวาน 2 ชิ้นเท่านั้น แต่ฟินง่ะ\n\nแอบอ่านรีวิวก่อนหน้า ส่วนใหญ่จะ complain อาหารคาว',
  'text': {'keywords': 'ช็อกโกแลต, เค้ก, ขนมหวาน'},
  'contexts': ['ช็อกโกแลต', 'เค้กช็อกโกแลตครีมเค้กช็อกโกแลตดำ', 'ขนมอร่อย']},
 {'review': 'ถ้ามีโอกาสมาสนามบิน ห้ามพลาด ร้านอาหารเล็กๆ มีให้เลือกทั้งก๋วยเตี๋ยว ส้มตำ น้ำตก อาหารจานเดียว หรือกาแฟสด น้ำผลไม้ต่างๆ ให้เลือกหลากหลาย อีกทั้งบรรยากาศดีเหมือนอยู่ในท้องนา ภายในสนามบินยังมีสินค้าจำหน่ายย่อมๆ ให้ซื้อเป็นที่ระลึก และยังมีสวนสัตว์ย่อมๆ ไว้ให้เพลิดเพลินด้วย',
  'text': {'keywords': 'สนามบิน, ร้านอาหาร, บรรยากาศ'},
  'contexts': ['สนามบิน', 'ร้านอาหารเล็กๆ', 'บรรยากาศดีเหมือนอยู่ในท้องนา']}]

In [122]:
sentiment_inputs = []

for review_data in tqdm(results_100, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 400/400 [00:00<00:00, 256101.60it/s]


In [123]:
len(sentiment_inputs)

1200

In [124]:
sentiment_inputs[:6]

[{'message': 'ช็อกโกแลต'},
 {'message': 'เค้กช็อกโกแลตครีมเค้กช็อกโกแลตดำ'},
 {'message': 'ขนมอร่อย'},
 {'message': 'สนามบิน'},
 {'message': 'ร้านอาหารเล็กๆ'},
 {'message': 'บรรยากาศดีเหมือนอยู่ในท้องนา'}]

In [125]:
for i in tqdm(range(len(results_100)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_100[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 400/400 [18:46<00:00,  2.82s/it]


In [126]:
results_100[0:2]

[{'review': 'Chocolate crape cake with strawberry 145฿\nDark chocolate Ganache cake 145฿\n\nบริการแบบ งงๆ แบบมีหลายเคาร์เตอร์ \n\nขนมอร่อย ร้านตกแต่งสวย มีทั้งอาหาร ขนม กาแฟ มีของ grocery ให้ซื้อไป cook เองที่บ้านได้ \n\nอิ่มคาวมาจากร้านข้างบนแล้ว มาต่อแค่ขนมหวาน 2 ชิ้นเท่านั้น แต่ฟินง่ะ\n\nแอบอ่านรีวิวก่อนหน้า ส่วนใหญ่จะ complain อาหารคาว',
  'text': {'keywords': 'ช็อกโกแลต, เค้ก, ขนมหวาน'},
  'contexts': ['ช็อกโกแลต', 'เค้กช็อกโกแลตครีมเค้กช็อกโกแลตดำ', 'ขนมอร่อย'],
  'sentiment': ['neutral', 'neutral', 'positive']},
 {'review': 'ถ้ามีโอกาสมาสนามบิน ห้ามพลาด ร้านอาหารเล็กๆ มีให้เลือกทั้งก๋วยเตี๋ยว ส้มตำ น้ำตก อาหารจานเดียว หรือกาแฟสด น้ำผลไม้ต่างๆ ให้เลือกหลากหลาย อีกทั้งบรรยากาศดีเหมือนอยู่ในท้องนา ภายในสนามบินยังมีสินค้าจำหน่ายย่อมๆ ให้ซื้อเป็นที่ระลึก และยังมีสวนสัตว์ย่อมๆ ไว้ให้เพลิดเพลินด้วย',
  'text': {'keywords': 'สนามบิน, ร้านอาหาร, บรรยากาศ'},
  'contexts': ['สนามบิน', 'ร้านอาหารเล็กๆ', 'บรรยากาศดีเหมือนอยู่ในท้องนา'],
  'sentiment': ['neutral', 'neutral', 'positive']}]

In [127]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_100
])

In [128]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')